In [53]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 크롤링할 URL 생성하는 함수 만들기(검색어, 날짜)
def makeUrl(search, date):
    formattedDate = date.replace("-", ".")
    url = f"https://search.naver.com/search.naver?where=news&query={search}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={formattedDate}&de={formattedDate}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{formattedDate}to{formattedDate}&is_sug_officeid=0&office_category=0&service_area=0"
    print("생성된 URL: ", url)
    return url

# HTML에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def newsAttrsCrawler(articles, attrs):
    attrsContent = []
    for i in articles:
        attrsContent.append(i.attrs[attrs])
    return attrsContent

# ConnectionError 방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# HTML 생성해서 기사 크롤링하는 함수 만들기(url): 링크를 반환
def articlesCrawler(url):
    # HTML 불러오기
    originalHtml = requests.get(url)
    html = BeautifulSoup(originalHtml.text, "html.parser")

    urlNaver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")  # CSS 선택자를 통해 뉴스 링크 요소 선택
    url = newsAttrsCrawler(urlNaver, 'href')  # 선택된 요소들 중 href(링크) 속성만 추출하는 함수 호출
    return url

##### 뉴스 크롤링 시작 #####

# 검색어 입력
search = input("검색할 지하철역을 입력해주세요:")
date = input("궁금하신 날짜를 입력해주세요 (YYYY-MM-DD)")

# Naver URL 생성
url = makeUrl(search, date)

# 뉴스 크롤러 실행
newsTitles = []
newsUrl = []
newsContents = []
newsDates = []

url = articlesCrawler(url)
newsUrl.append(url)

# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newList, content):
    for i in content:
        for j in i:
            newList.append(j)
    return newList

# 제목, 링크, 내용 담을 리스트 생성
newsUrl1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(newsUrl1, newsUrl)

# 신뢰도를 위해 NAVER 뉴스만 남기기
finalUrls = []
for i in tqdm(range(len(newsUrl1))):
    if "news.naver.com" in newsUrl1[i]:
        finalUrls.append(newsUrl1[i])
    else:
        pass

# 뉴스 내용 크롤링 (진행률 표시)
for i in tqdm(finalUrls):
    # 각 기사 HTML get하기
    news = requests.get(i, headers=headers)
    newsHtml = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = newsHtml.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title is None:
        title = newsHtml.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = newsHtml.select("article#dic_area")
    if content == []:
        content = newsHtml.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list 합치기
    content = ''.join(str(content))

    # 정규 표현식 활용 HTML 태그 제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    content = re.sub(r'\n+', ' ', content)
    newsTitles.append(title)
    newsContents.append(content)

    try:
        htmlDate = newsHtml.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        newsDate = htmlDate.attrs['data-date-time']
    except AttributeError:
        newsDate = newsHtml.select_one("#content > div.end_ct > div > div.article_info > span > em")
        newsDate = re.sub(pattern=pattern1, repl='', string=str(newsDate))
    # 날짜 가져오기
    newsDates.append(newsDate)

print("\n[뉴스 제목]")
print(newsTitles)
print("\n[뉴스 링크]")
print(finalUrls)
print("\n[뉴스 내용]")
print(newsContents)

print('newsTitle: ', len(newsTitles))
print('newsUrl: ', len(finalUrls))
print('newsContents: ', len(newsContents))
print('newsDates: ', len(newsDates))

### 데이터 프레임으로 만들기 ###
import pandas as pd

# 데이터 프레임 만들기
newsDf = pd.DataFrame({'date': newsDates, 'title': newsTitles, 'link': finalUrls, 'content': newsContents})

# 중복 행 지우기
newsDf = newsDf.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(newsDf))

# 데이터 프레임 저장
now = datetime.datetime.now()
newsDf['date'] = pd.to_datetime(newsDf['date']).dt.strftime('%Y-%m-%d')  # 날짜 형식 변경
filename = '{}_{}.csv'.format(search, date)
newsDf.to_csv(filename, encoding='utf-8-sig', index=False)


검색할 지하철역을 입력해주세요:증산역
궁금하신 날짜를 입력해주세요 (YYYY-MM-DD)2023-03-15
생성된 URL:  https://search.naver.com/search.naver?where=news&query=증산역&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.03.15&de=2023.03.15&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom2023.03.15to2023.03.15&is_sug_officeid=0&office_category=0&service_area=0


100%|██████████| 1/1 [00:00<00:00,  2.05it/s]


[뉴스 제목]
['오늘 새벽 서울 증산역 인근서 마주오던 차량 충돌']

[뉴스 링크]
['https://n.news.naver.com/mnews/article/056/0011445529?sid=102']

[뉴스 내용]
["[ 오늘(15일) 새벽 1시 반쯤 서울 은평구 증산역 인근에서 마주 오던 승합차와 승용차가 부딪치는 사고가 일어났습니다. 이 사고로 두 차량의 운전자가 각각 의식이 있는 상태로 병원으로 옮겨졌습니다. 경찰은 운전자들에 대해 음주 여부를 검사했지만, 음주운전은 아니었던 것으로 파악됐습니다. 경찰은 정확한 사고 경위를 조사하고 있습니다.[사진 출처 : 시청자 강대성 님 제공]■ 제보하기▷ 카카오톡 : 'KBS제보' 검색▷ 전화 : 02-781-1234▷ 이메일 : kbs1234@kbs.co.kr▷ 뉴스홈페이지 : https://goo.gl/4bWbkG ]"]
newsTitle:  1
newsUrl:  1
newsContents:  1
newsDates:  1
중복 제거 후 행 개수:  1
